<a href="https://colab.research.google.com/github/OlegPM/UFC-predictor/blob/main/Copy_of_UFC_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [2]:
#Loading all csv files in data frames
ufc_fighter_tott_df = pd.read_csv('ufc_fighter_tott.csv', sep=",")
ufc_event_details_df = pd.read_csv('ufc_event_details.csv', sep=",")
ufc_fight_results_df = pd.read_csv('ufc_fight_results.csv', sep=",")

In [3]:
# Create a new DataFrame with the split columns
new_df = ufc_fight_results_df["BOUT"].str.split(" vs. ", n=1, expand=True)

# Rename the columns
new_df.columns = ["FIGHTER1", "FIGHTER2"]

insert_idx = ufc_fight_results_df.columns.get_loc('BOUT')+1

# insert the new DataFrame at the specified index
ufc_fight_results_df = pd.concat([ufc_fight_results_df.iloc[:, :insert_idx], new_df, ufc_fight_results_df.iloc[:, insert_idx:]], axis=1)


# Drop the original "BOUT" "URL" "DETAILS" column
ufc_fight_results_df.drop("BOUT", axis=1, inplace=True)
ufc_fight_results_df.drop("URL", axis=1, inplace=True)
ufc_fight_results_df.drop("DETAILS", axis=1, inplace=True)
ufc_fight_results_df.drop("REFEREE", axis=1, inplace=True)
ufc_fight_results_df.drop("METHOD", axis=1, inplace=True)

# select the rows with the specific values in the "TIME FORMAT" column
time_formats = ['5 Rnd (5-5-5-5-5)', '3 Rnd (5-5-5)']
ufc_fight_results_df = ufc_fight_results_df[ufc_fight_results_df['TIME FORMAT'].isin(time_formats)]

In [4]:
ufc_weight_classes = [    
    "Bantamweight Bout",    
    "Featherweight Bout",  
    "Flyweight Bout", 
    "Heavyweight Bout", 
    "Light Heavyweight Bout",    
    "Lightweight Bout",    
    "Middleweight Bout",

    "UFC Bantamweight Title Bout", 
    "UFC Featherweight Title Bout",   
    "UFC Flyweight Title Bout",   
    "UFC Heavyweight Title Bout",
    "UFC Light Heavyweight Title Bout",
    "UFC Lightweight Title Bout",    
    "UFC Middleweight Title Bout",  
    "UFC Welterweight Title Bout",    
      
    "UFC Women's Bantamweight Title Bout", 
    "UFC Women's Featherweight Title Bout",   
    "UFC Women's Flyweight Title Bout", 
    "UFC Women's Strawweight Title Bout",

    "Welterweight Bout",

    "Women's Bantamweight Bout",    
    "Women's Featherweight Bout", 
    "Women's Flyweight Bout",
    "Women's Strawweight Bout"
    ]
None

In [5]:
#Deletes all WEIGHTCLASSes beside those which are in the ufc_weight_classes
ufc_fight_results_df = ufc_fight_results_df[ufc_fight_results_df['WEIGHTCLASS'].isin(ufc_weight_classes)];

In [6]:
ufc_event_details_df.drop("LOCATION", axis=1, inplace=True)
ufc_event_details_df.drop("URL", axis=1, inplace=True)

In [7]:
# Remove trailing whitespace from 'EVENT' column in both dataframes
ufc_fight_results_df['EVENT'] = ufc_fight_results_df['EVENT'].str.strip()
ufc_event_details_df['EVENT'] = ufc_event_details_df['EVENT'].str.strip()

# Perform left join on 'EVENT' column
ufc_fight_results_df = pd.merge(ufc_fight_results_df, ufc_event_details_df[['EVENT', 'DATE']], on='EVENT', how='left')
None

In [8]:
ufc_fighter_tott_df = ufc_fighter_tott_df.replace("--", np.nan)
ufc_fighter_tott_df.drop("URL", axis=1, inplace=True)
None

In [9]:
# Define dictionary of column name mappings
mapping = {col: col.replace('.', '').replace(' ', '_').lower() for col in ufc_fight_results_df.columns}
mapping = {col: col.replace('.', '').replace(' ', '_').lower() for col in ufc_fighter_tott_df.columns}
# Rename columns with dictionary of mappings
ufc_fight_results_df = ufc_fight_results_df.rename(columns=mapping)
ufc_fighter_tott_df = ufc_fighter_tott_df.rename(columns=mapping)

In [10]:
# drop any rows with missing values
ufc_fight_results_df = ufc_fight_results_df.dropna(how='any')
ufc_fighter_tott_df = ufc_fighter_tott_df.dropna(how='any')

In [11]:
def ft_in_to_cm(height_str):
    feet, inches = height_str.split('\'')
    total_inches = int(feet) * 12 + int(inches.strip('"'))
    return round(total_inches * 2.54, 1)

cat_list = ['stance']
for col in cat_list:
    # perform one hot encoding using get_dummies() function
    encoded = pd.get_dummies(ufc_fighter_tott_df[col], prefix=col)

    # concatenate the encoded dataframe with the original dataframe
    ufc_fighter_tott_df = pd.concat([ufc_fighter_tott_df, encoded], axis=1)

    # drop the original categorical column A
    ufc_fighter_tott_df = ufc_fighter_tott_df.drop(col, axis=1)

col_list = ["str_acc", "str_def", "td_acc", "td_def"]
for col in col_list:
    ufc_fighter_tott_df[col] = ufc_fighter_tott_df[col].str.rstrip('%').astype(float) / 100

col_list = ["height"]
for col in col_list:
    # Apply the function to the column and create a new column with the result
    ufc_fighter_tott_df[col] = ufc_fighter_tott_df[col].apply(ft_in_to_cm)

col_list = ["reach"]
for col in col_list:
    ufc_fighter_tott_df[col] = ufc_fighter_tott_df[col].str.replace("\"", "").astype(int) * 2.54

col_list = ["weight"]
for col in col_list:
    ufc_fighter_tott_df[col] = ufc_fighter_tott_df[col].str.replace(" lbs.", "", regex=True).astype(int)


In [12]:
# Remove trailing whitespace from 'EVENT' column in both dataframes
ufc_fight_results_df['FIGHTER1'] = ufc_fight_results_df['FIGHTER1'].str.strip()
ufc_fight_results_df['FIGHTER2'] = ufc_fight_results_df['FIGHTER2'].str.strip()
ufc_fighter_tott_df['fighter'] = ufc_fighter_tott_df['fighter'].str.strip()

# Perform left join on 'FIGHTER1' column and 'FIGHTER' column

tmp_df = ufc_fighter_tott_df.add_suffix('_fighter1')
ufc_fight_results_df = ufc_fight_results_df.merge(tmp_df, left_on='FIGHTER1', right_on='fighter_fighter1')
tmp_df = ufc_fighter_tott_df.add_suffix('_fighter2')
ufc_fight_results_df = ufc_fight_results_df.merge(tmp_df, left_on='FIGHTER2', right_on='fighter_fighter2')
ufc_fight_results_df = ufc_fight_results_df.drop(['fighter_fighter1','fighter_fighter2'], axis=1)


In [13]:
cat_list = ['OUTCOME','TIME FORMAT','WEIGHTCLASS']
for col in cat_list:
    # create dummy variables for column outcome
    dummies = pd.get_dummies(ufc_fight_results_df[col])

    # add the dummy variables to the original dataframe
    ufc_fight_results_df = pd.concat([ufc_fight_results_df, dummies], axis=1)

    # drop the original categorical column A
    ufc_fight_results_df = ufc_fight_results_df.drop(col, axis=1)

In [14]:
# convert the birth date column to datetime
ufc_fight_results_df['dob_fighter1'] = pd.to_datetime(ufc_fight_results_df['dob_fighter1'], format='%b %d, %Y')
ufc_fight_results_df['dob_fighter2'] = pd.to_datetime(ufc_fight_results_df['dob_fighter2'], format='%b %d, %Y')

# convert the date column to datetime
ufc_fight_results_df['DATE'] = pd.to_datetime(ufc_fight_results_df['DATE'], format='%B %d, %Y')

# subtract the birth date from the event date to get the age
ufc_fight_results_df['age_fighter1'] = (ufc_fight_results_df['DATE'] - pd.to_datetime(ufc_fight_results_df['dob_fighter1'], format='%b %d, %Y')).dt.days / 365.25
ufc_fight_results_df['age_fighter2'] = (ufc_fight_results_df['DATE'] - pd.to_datetime(ufc_fight_results_df['dob_fighter2'], format='%b %d, %Y')).dt.days / 365.25
ufc_fight_results_df.head()

,EVENT,FIGHTER1,FIGHTER2,ROUND,TIME,DATE,height_fighter1,weight_fighter1,reach_fighter1,dob_fighter1,slpm_fighter1,str_acc_fighter1,sapm_fighter1,str_def_fighter1,td_avg_fighter1,td_acc_fighter1,td_def_fighter1,sub_avg_fighter1,win_fighter1,lose_fighter1,draw_fighter1,stance_Open Stance_fighter1,stance_Orthodox_fighter1,stance_Southpaw_fighter1,stance_Switch_fighter1,height_fighter2,weight_fighter2,reach_fighter2,dob_fighter2,slpm_fighter2,str_acc_fighter2,sapm_fighter2,str_def_fighter2,td_avg_fighter2,td_acc_fighter2,td_def_fighter2,sub_avg_fighter2,win_fighter2,lose_fighter2,draw_fighter2,stance_Open Stance_fighter2,stance_Orthodox_fighter2,stance_Southpaw_fighter2,stance_Switch_fighter2,D/D,L/W,NC/NC,W/L,3 Rnd (5-5-5),5 Rnd (5-5-5-5-5),Bantamweight Bout,Featherweight Bout,Flyweight Bout,Heavyweight Bout,Light Heavyweight Bout,Lightweight Bout,Middleweight Bout,UFC Bantamweight Title Bout,UFC Featherweight Title Bout,UFC Flyweight Title Bout,UFC Heavyweight Title Bout,UFC Light Heavyweight Title Bout,UFC Lightweight Title Bout,UFC Middleweight Title Bout,UFC Welterweight Title Bout,UFC Women's Bantamweight Title Bout,UFC Women's Featherweight Title Bout,UFC Women's Flyweight Title Bout,UFC Women's Strawweight Title Bout,Welterweight Bout,Women's Bantamweight Bout,Women's Featherweight Bout,Women's Flyweight Bout,Women's Strawweight Bout,age_fighter1,age_fighter2
0,UFC Fight Night: Vera vs. Sandhagen,Marlon Vera,Cory Sandhagen,5,5:00,2023-03-25,172.7,135,177.80,1992-12-02,4.09,0.49,5.02,0.51,0.61,0.40,0.69,1.0,20,8,1,0,0,0,1,180.3,135,177.8,1992-04-20,5.98,0.44,3.9,0.57,0.89,0.21,0.64,0.3,16,4,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30.308008,30.926762
1,UFC 241: Cormier vs. Miocic 2,Raphael Assuncao,Cory Sandhagen,3,5:00,2019-08-17,165.1,135,167.64,1982-07-19,3.22,0.42,2.50,0.66,1.65,0.39,0.78,0.2,28,10,0,0,1,0,0,180.3,135,177.8,1992-04-20,5.98,0.44,3.9,0.57,0.89,0.21,0.64,0.3,16,4,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.078713,27.323751
2,UFC 250: Nunes vs. Spencer,Aljamain Sterling,Cory Sandhagen,1,1:28,2020-06-06,170.2,135,180.34,1989-07-31,4.70,0.51,2.18,0.61,1.98,0.24,0.41,0.9,22,3,0,0,1,0,0,180.3,135,177.8,1992-04-20,5.98,0.44,3.9,0.57,0.89,0.21,0.64,0.3,16,4,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30.850103,28.128679
3,UFC Fight Night: Moraes vs. Sandhagen,Marlon Moraes,Cory Sandhagen,2,1:03,2020-10-10,167.6,135,170.18,1988-04-26,3.37,0.40,4.53,0.54,0.87,0.41,0.46,0.3,23,10,1,0,1,0,0,180.3,135,177.8,1992-04-20,5.98,0.44,3.9,0.57,0.89,0.21,0.64,0.3,16,4,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32.457221,28.473648
4,UFC Fight Night: Jacare vs. Hermansson,John Lineker,Cory Sandhagen,3,5:00,2019-04-27,160.0,135,170.18,1990-06-12,5.36,0.40,4.27,0.56,0.56,0.58,0.67,0.6,31,9,0,0,1,0,0,180.3,135,177.8,1992-04-20,5.98,0.44,3.9,0.57,0.89,0.21,0.64,0.3,16,4,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.873374,27.017112


In [15]:
# split the time column into minutes and seconds
ufc_fight_results_df[['minutes', 'seconds']] = ufc_fight_results_df['TIME'].str.split(':', expand=True).astype(int)

# calculate the total duration in seconds
ufc_fight_results_df['duration_seconds'] = (ufc_fight_results_df['ROUND'] - 1) * 5 * 60 + ufc_fight_results_df['minutes'] * 60 + ufc_fight_results_df['seconds']

# convert the duration to minutes
ufc_fight_results_df['duration_minutes'] = ufc_fight_results_df['duration_seconds'] / 60

# drop the temporary columns
ufc_fight_results_ready_to_use_df = ufc_fight_results_df.drop(['minutes', 'seconds', 'duration_seconds', 'ROUND', 'TIME', 'FIGHTER1', 'FIGHTER2', 'EVENT', 'DATE', 'dob_fighter1', 'dob_fighter2'], axis=1)


In [16]:
# # Logistic Regression
# # create an instance of StandardScaler
# scaler = StandardScaler()

# # Separate the target variable (D/D, L/W, NC/NC, W/L) from the features
# target = ufc_fight_results_ready_to_use_df[['W/L']]
# features = ufc_fight_results_ready_to_use_df.drop(['W/L'], axis=1)


# # One-hot encode the categorical variables in the target variable
# target = pd.get_dummies(target)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# y_train = np.ravel(y_train)

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)


# # Train a logistic regression model on the training data
# clf = LogisticRegression()
# clf.fit(X_train, y_train)

# # Make predictions on the test data
# y_pred = clf.predict(X_test)

# # Calculate the accuracy of the model
# accuracy = accuracy_score(y_test, y_pred)
# print('Logistic Regression')
# print('Accuracy:', accuracy)

In [17]:
# # Logistic Regression with Cross validation
# # create an instance of StandardScaler
# scaler = StandardScaler()

# # fit and transform the data

# # Separate the target variable (D/D, L/W, NC/NC, W/L) from the features
# target = ufc_fight_results_ready_to_use_df[['W/L']]
# features = ufc_fight_results_ready_to_use_df.drop(['W/L'], axis=1)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# y_train = np.ravel(y_train)

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)

# # Train a logistic regression model using cross-validation
# clf = LogisticRegression()
# scores = cross_val_score(clf, X_train, y_train, cv=5)
# print('Logistic Regression with Cross validation')
# print('Accuracy Scores:', scores)
# print('Mean Accuracy:', scores.mean())

In [18]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import StandardScaler

# # Separate the target variable (D/D, L/W, NC/NC, W/L) from the features
# target = ufc_fight_results_ready_to_use_df[['W/L']]
# features = ufc_fight_results_ready_to_use_df.drop(['W/L'], axis=1)

# # One-hot encode the categorical variables in the target variable
# target = pd.get_dummies(target)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# # Create an instance of StandardScaler and fit/transform the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# y_train = np.ravel(y_train)

# # Create an instance of RandomForestClassifier and train it on the training data
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf.fit(X_train, y_train)

# # Make predictions on the test data
# y_pred = clf.predict(X_test)

# # Calculate the accuracy of the model
# accuracy = accuracy_score(y_test, y_pred)
# print('Accuracy:', accuracy)



In [19]:
# #!pip install tensorflow

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# # Separate the target variable (D/D, L/W, NC/NC, W/L) from the features
# target = ufc_fight_results_ready_to_use_df[['W/L']]
# features = ufc_fight_results_ready_to_use_df.drop(['W/L'], axis=1)

# # One-hot encode the categorical variables in the target variable
# target = pd.get_dummies(target)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Build the neural network model
# model = Sequential()
# model.add(Dense(units=32, activation='relu', input_dim=X_train.shape[1]))
# model.add(Dense(units=16, activation='relu'))
# model.add(Dense(units=y_train.shape[1], activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# # Evaluate the model on the test set
# loss, accuracy = model.evaluate(X_test, y_test)
# print('Test accuracy:', accuracy)


In [20]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.metrics import Precision, Recall, AUC


target = ufc_fight_results_ready_to_use_df[['D/D', 'L/W', 'NC/NC', 'W/L']]
features = ufc_fight_results_ready_to_use_df.drop(['D/D', 'L/W', 'NC/NC', 'W/L'], axis=1)

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, test_size=0.2, random_state=42)

# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[AUC()])

# Train the model
history = model.fit(X_train, y_train, epochs=40, validation_split=0.2)

# Evaluate the model on the test set
y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test.values, axis=1)
accuracy = accuracy_score(y_test_classes, y_pred_classes)
print('Accuracy:', accuracy)


Epoch 1/40
118/118 [==============================] - 2s 7ms/step - loss: 0.9317 - auc: 0.8342 - val_loss: 0.7133 - val_auc: 0.8885
Epoch 2/40
118/118 [==============================] - 0s 3ms/step - loss: 0.7367 - auc: 0.8870 - val_loss: 0.6871 - val_auc: 0.8964
Epoch 3/40
118/118 [==============================] - 1s 5ms/step - loss: 0.6984 - auc: 0.8997 - val_loss: 0.6760 - val_auc: 0.9007
Epoch 4/40
118/118 [==============================] - 0s 4ms/step - loss: 0.6793 - auc: 0.9063 - val_loss: 0.6635 - val_auc: 0.9058
Epoch 5/40
118/118 [==============================] - 0s 3ms/step - loss: 0.6568 - auc: 0.9132 - val_loss: 0.6584 - val_auc: 0.9074
Epoch 6/40
118/118 [==============================] - 0s 2ms/step - loss: 0.6404 - auc: 0.9172 - val_loss: 0.6556 - val_auc: 0.9090
Epoch 7/40
118/118 [==============================] - 0s 2ms/step - loss: 0.6434 - auc: 0.9161 - val_loss: 0.6504 - val_auc: 0.9102
Epoch 8/40
118/118 [==============================] - 0s 2ms/step - loss: 0.

In [29]:
import numpy as np
import ipywidgets as widgets
from datetime import datetime
from IPython.display import display

from IPython.display import clear_output


# Define the features for the two fighters
# Create a dropdown widget

# Group fighters by weight class
fighters = sorted(list(ufc_fighter_tott_df['fighter'].unique()))

fighter_options = {}

# Create a dropdown for weight class
weight_dropdown = widgets.Dropdown(
    options = ufc_weight_classes,
    value = ufc_weight_classes[0],
    description = 'Weight Class:'
)

# Create a dropdown for the number of rounds
rounds_dropdown = widgets.Dropdown(
options=[('3 Rnd', 0), ('5 Rnd', 1)],
value=0,
description='Number of Rounds:'
)

# Create a dropdown for fighters
fighter_dropdown1 = widgets.Dropdown(
    options = list(fighters),
    value = list(fighters)[0],
    description='Fighter1:'
)

fighter_dropdown2 = widgets.Dropdown(
    options = list(fighters),
    value = list(fighters)[0],
    description='Fighter2:'
)

# Set default date to today
default_date = datetime.today().date()

# Create a DatePicker widget
event_date_picker = widgets.DatePicker(
    description='Event date',
    value=default_date
)

# Define a function to predict the an outcome
def predict_outcome(button):
    # Get the selected fighter1 from the dropdown
    selected_fighter1 = fighter_dropdown1.value
    
    # Filter the dataframe by the selected fighter1
    fighter1_data = ufc_fighter_tott_df.loc[ufc_fighter_tott_df['fighter'] == selected_fighter1]

    # Get the selected fighter2 from the dropdown
    selected_fighter2 = fighter_dropdown2.value

    # Filter the dataframe by the selected fighter2
    fighter2_data = ufc_fighter_tott_df.loc[ufc_fighter_tott_df['fighter'] == selected_fighter2]

    # Create a Series object with all the features
    fighter1_series = pd.Series(fighter1_data.iloc[0])
    fighter2_series = pd.Series(fighter2_data.iloc[0])

    # calculate age of fighter 1
    
    event_date_str = event_date_picker.value.strftime('%Y-%m-%d')
    event_date = datetime.strptime(event_date_str, '%Y-%m-%d').date()

    fighter1_dob = datetime.strptime(fighter1_series['dob'], '%b %d, %Y').date()
    fighter2_dob = datetime.strptime(fighter2_series['dob'], '%b %d, %Y').date()

    age_fighter1 = (event_date - fighter1_dob).days // 365
    age_fighter2 = (event_date - fighter2_dob).days // 365

    age_fighter1_series = pd.Series(dtype='float64')
    age_fighter2_series = pd.Series(dtype='float64')

    age_fighter1_series['age_fighter1'] = age_fighter1
    age_fighter2_series['age_fighter2'] = age_fighter2

    fighter1_series = fighter1_series.drop(['fighter', 'dob'])
    fighter2_series = fighter2_series.drop(['fighter', 'dob'])

    fighter1_series = fighter1_series.add_suffix('_fighter1')
    fighter2_series = fighter2_series.add_suffix('_fighter2')

    


    # Add the new fields to the tmp_series

    tmp_series = pd.Series(dtype='float64')
    tmp_series['3 Rnd (5-5-5)']      = 0
    tmp_series['5 Rnd (5-5-5-5-5)']  = 0

    duration_minutes_series = pd.Series(dtype='float64')
    

    # Set the value of the selected rounds in the respective fighter's series
    if rounds_dropdown.value == 0:
          tmp_series['3 Rnd (5-5-5)'] = 1
          duration_minutes_series['duration_minutes'] = 15
    else:
          tmp_series['5 Rnd (5-5-5-5-5)'] = 1
          duration_minutes_series['duration_minutes'] = 25
    
    # Create a Series with fields from ufc_weight_classes
    # weight_class_series = pd.Series(index=ufc_weight_classes)
    weight_class_series = pd.Series({weight_class: 1 if weight_class == weight_dropdown.value else 0 for weight_class in ufc_weight_classes}, dtype='uint8')

    # Define a function to update the weight class series
    def update_weight_class_series(weight_class):
        weight_class_series[weight_class] = 1

    # Link the weight dropdown to the update function
    weight_dropdown.observe(lambda change: update_weight_class_series(change.new), 'value')




    combined_series = pd.concat([fighter1_series, fighter2_series, tmp_series, weight_class_series, age_fighter1_series, age_fighter2_series, duration_minutes_series])
    combined_df = pd.DataFrame([combined_series])

    data_scaled = scaler.transform(combined_df)

    prediction_percent = model.predict(data_scaled)[0] * 100
    print(f"Format: {rounds_dropdown.options[rounds_dropdown.value][0]}")
    print(f"Date: {event_date_picker.value}")
    print(f"{fighter_dropdown1.value} win probability: {prediction_percent[3]:.2f}%")
    print(f"{fighter_dropdown2.value} win probability: {prediction_percent[1]:.2f}%")
    print(f"DRAW probability: {prediction_percent[0]:.2f}%")
    print(f"NC probability: {prediction_percent[2]:.2f}%")
    

    return None

# Create a button and add the print_fighter_name function as an on_click event
button = widgets.Button(description='Predict')
button.on_click(predict_outcome)

# Display the widgets and button
display(weight_dropdown, rounds_dropdown, event_date_picker, fighter_dropdown1, fighter_dropdown2, button)



Dropdown(description='Weight Class:', options=('Bantamweight Bout', 'Featherweight Bout', 'Flyweight Bout', 'H…

Dropdown(description='Number of Rounds:', options=(('3 Rnd', 0), ('5 Rnd', 1)), value=0)

DatePicker(value=datetime.date(2023, 4, 4), description='Event date')

Dropdown(description='Fighter1:', options=('AJ Dobson', 'AJ Fletcher', 'Aalon Cruz', 'Aaron Jeffery', 'Aaron P…

Dropdown(description='Fighter2:', options=('AJ Dobson', 'AJ Fletcher', 'Aalon Cruz', 'Aaron Jeffery', 'Aaron P…

Button(description='Predict', style=ButtonStyle())

1/1 [==============================] - 0s 24ms/step
Format: 5 Rnd
Date: 2023-04-04
Tony Ferguson win probability: 34.71%
Sean O'Malley win probability: 65.28%
DRAW probability: 0.00%
NC probability: 0.01%
1/1 [==============================] - 0s 17ms/step
Format: 5 Rnd
Date: 2023-04-04
Tony Ferguson win probability: 49.95%
Sean O'Malley win probability: 49.98%
DRAW probability: 0.00%
NC probability: 0.06%
1/1 [==============================] - 0s 19ms/step
Format: 5 Rnd
Date: 2023-04-04
Tony Ferguson win probability: 9.03%
Sean O'Malley win probability: 90.96%
DRAW probability: 0.00%
NC probability: 0.01%
1/1 [==============================] - 0s 18ms/step
Format: 3 Rnd
Date: 2023-04-04
Tony Ferguson win probability: 25.51%
Sean O'Malley win probability: 74.46%
DRAW probability: 0.00%
NC probability: 0.02%
